#  Sentiement Analysis

Context

This is the sentiment140 dataset. It contains 1,600,000 tweets extracted using the twitter api . The tweets have been annotated (0 = negative, 4 = positive) and they can be used to detect sentiment .
Content

It contains the following 6 fields:

  - target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

  - ids: The id of the tweet ( 2087)

  - date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)

  - flag: The query (lyx). If there is no query, then this value is NO_QUERY.

  - user: the user that tweeted (robotickilldozr)

  - text: the text of the tweet (Lyx is cool)


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.text import Text
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# I- Data Preparation

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')




Mounted at /content/gdrive/


In [ ]:
import requests
import json
api_key='q1HJoEmRMdypie_E6lYwiih0hNfuSatSTQXH6Bh9fnltm30JX4XO1TSCd6NARRVszrs4lMXb7N3i6LWB_UIBEinyG_ah-0expG6GUnzTdUss2gvNQRcJ7fJhK5g2YHYx'
headers = {'Authorization': 'Bearer %s' % api_key}


url = "https://api.yelp.com/v3/businesses/FEVQpbOPOwAPNIgO7D3xxw/reviews"
req = requests.get(url, headers=headers)
print('the status code is {}'.format(req.status_code))


data=json.loads(req.text)
data

the status code is 200


{'possible_languages': ['fr', 'en', 'zh', 'pt', 'de', 'it', 'sv', 'ja', 'es'],
 'reviews': [{'id': 'qOiuCTz_Gzgmose1GHaAlg',
   'rating': 5,
   'text': "Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...",
   'time_created': '2020-12-19 07:52:35',
   'url': 'https://www.yelp.com/biz/shake-shack-new-york-2?adjust_creative=KzZVmcfbYqCMY9omoha5NA&hrid=qOiuCTz_Gzgmose1GHaAlg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KzZVmcfbYqCMY9omoha5NA',
   'user': {'id': 'FRKnLBnlFiasr1Dc8oOIGQ',
    'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/1wm0yjWSw92j_ZsUFZBGzQ/o.jpg',
    'name': 'Sarah G.',
    'profile_url': 'https://www.yelp.com/user_details?userid=FRKnLBnlFiasr1Dc8oOIGQ'}},
  {'id': '7d56A_ObMPHHyywcfhnrUw',
   'rating': 5,
   'text': 'Happened to be in the city today and not too far from here so I had to stop by and pick up cheese fries (I love those c

In [ ]:
text  = data["reviews"][0]["text"]
print(text)
print(type(text))

Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...
<class 'str'>


## B - Clean data 

### Tokenization

In [ ]:
from time import time
from nltk.tokenize import TweetTokenizer 
start_time = time()
tokenizer  = TweetTokenizer(reduce_len = True)
#convert dataframe to list
data = []

#Create a X training matrix and a Y label matrix 
X_text_list = [text]


# Convert label value "4: positive or 1: negative" to 0 or 1 : binary values
# 1 : postive comment
# 0 : negative comment
for text in X_text_list:
    data.append((tokenizer.tokenize(text)))
print("Computation Time", time() - start_time)
data[:3]

Computation Time 0.0016748905181884766


[['Truth',
  'be',
  'told',
  'if',
  'it',
  'was',
  'up',
  'to',
  'me',
  "I'd",
  'be',
  'giving',
  '4/5',
  'stars',
  ',',
  'we',
  'did',
  'order',
  'recently',
  'and',
  'our',
  'cheese',
  'fries',
  "weren't",
  'delivered',
  ',',
  "couldn't",
  'speak',
  'with',
  'anyone',
  'from',
  '...']]

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

### Lemmatization

In [ ]:
from nltk.tag  import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

def lemmatize_token(tokens):
  """
  take a token in input return its lemmatize version
  """
  lemmatizer = WordNetLemmatizer()
  lemmatized_token = []
  for word, tag in pos_tag(tokens):
    if (tag.startswith('NN')):
      pos = 'n'
    elif (tag.startswith('VB')):
      pos = 'v'
    else:
      pos = 'a'
    lemmatized_token.append(lemmatizer.lemmatize(word, pos))
  return lemmatized_token

print(lemmatize_token(data[0]))

['Truth', 'be', 'tell', 'if', 'it', 'be', 'up', 'to', 'me', "I'd", 'be', 'give', '4/5', 'star', ',', 'we', 'do', 'order', 'recently', 'and', 'our', 'cheese', 'fry', "weren't", 'deliver', ',', "couldn't", 'speak', 'with', 'anyone', 'from', '...']


### Remove stopword


In [ ]:
import re, string
from nltk.corpus import stopwords
STOPWORD = stopwords.words('english')

def cleaner(token):
    if token == 'u':
      return 'you'
    if token == 'r':
        return 'are'
    if token == 'some1':
        return 'someone'
    if token == 'yrs':
        return 'years'
    if token == 'hrs':
        return 'hours'
    if token == 'mins':
        return 'minutes'
    if token == 'secs':
        return 'seconds'
    if token == 'pls' or token == 'plz':
        return 'please'
    if token == '2morow':
        return 'tomorrow'
    if token == '2day':
        return 'today'
    if token == '4got' or token == '4gotten':
        return 'forget'
    if token == 'amp' or token == 'quot' or token == 'lt' or token == 'gt' or token == '½25':
        return ''
    return token


def remove_noise(tokens):

  cleaned_tokens = []

  for token, tag in pos_tag(tokens):
      # Eliminating the token if it is a link
      token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                      '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
      # Eliminating the token if it is a mention
      token = re.sub("(@[A-Za-z0-9_]+)","", token)

      if tag.startswith("NN"):
          pos = 'n'
      elif tag.startswith('VB'):
          pos = 'v'
      else:
          pos = 'a'

      lemmatizer = WordNetLemmatizer()
      token = lemmatizer.lemmatize(token, pos)

      cleaned_token = cleaner(token.lower())
      
      # Eliminating the token if its length is less than 3, if it is a punctuation or if it is a stopword
      if cleaned_token not in string.punctuation and len(cleaned_token) > 2 and cleaned_token not in STOPWORD:
          cleaned_tokens.append(cleaned_token)
          
  return cleaned_tokens


print(remove_noise(data[0]))

['truth', 'tell', "i'd", 'give', '4/5', 'star', 'order', 'recently', 'cheese', 'fry', 'deliver', 'speak', 'anyone', '...']


In [ ]:
start_time = time()


# we have to define a function that transforms our data into the required input structure
def list_to_dict(cleaned_tokens):
    return dict([token, True] for token in cleaned_tokens)

cleaned_tokens_list = []

# Removing noise from all the data
for tokens in data:
    cleaned_tokens_list.append((remove_noise(tokens)))

print('Removed Noise, CPU Time:', time() - start_time)
start_time = time()

final_data = []

# Transforming the data to fit the input structure of the Naive Bayesian classifier
for tokens in cleaned_tokens_list:
    final_data.append((list_to_dict(tokens)))
    
print('Data Prepared for model, CPU Time:', time() - start_time)

# Previewing our final (tokenized, cleaned and lemmatized) data list
print(final_data[:5])
print(cleaned_tokens_list)

Removed Noise, CPU Time: 0.007508277893066406
Data Prepared for model, CPU Time: 0.00015878677368164062
[{'truth': True, 'tell': True, "i'd": True, 'give': True, '4/5': True, 'star': True, 'order': True, 'recently': True, 'cheese': True, 'fry': True, 'deliver': True, 'speak': True, 'anyone': True, '...': True}]
[['truth', 'tell', "i'd", 'give', '4/5', 'star', 'order', 'recently', 'cheese', 'fry', 'deliver', 'speak', 'anyone', '...']]


### Word embedding using GloVe
first I downloaded the glovefile text on kaggle and put it on my GoogleDrive

In [ ]:
# Defining a handy function in order to load a given glove file
import numpy as np
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [ ]:
# Loading the 50-dimensional GloVe embeddings
# This method will return three dictionaries:
# * word_to_index: a dictionary mapping from words to their indices in the vocabulary
# * index_to_word: dictionary mapping from indices to their corresponding words in the vocabulary
# * word_to_vec_map: dictionary mapping words to their GloVe vector representation
# Note that there are 400,001 words, with the valid indices ranging from 0 to 400,000

word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('/content/gdrive/MyDrive/Etudes/Natural_Language_Processing/SentimentAnalysis/glove.6B.100d.txt')

In [ ]:
print("word to index", word_to_index['hello'])
print("word to vec map", word_to_vec_map['hello'])


In [ ]:
start_time = time()

unks = []
UNKS = []

# This function will act as a "last resort" in order to try and find the word
# in the words embedding layer. It will basically eliminate contiguously occuring
# instances of a similar character
def cleared(word):
    res = ""
    prev = None
    for char in word:
        if char == prev: continue
        prev = char
        res += char
    return res


def sentence_to_indices(sentence_words, word_to_index, max_len, i):
    global X, Y
    sentence_indices = []
    for j, w in enumerate(sentence_words):
        try:
            index = word_to_index[w]
        except:
            UNKS.append(w)
            w = cleared(w)
            try:
                index = word_to_index[w]
            except:
                index = word_to_index['unk']
                unks.append(w)
        X[i, j] = index

        
# Here we will utilize the already computed 'cleaned_tokens_list' variable
   
print('Removed Noise, CPU Time:', time() - start_time)
start_time = time()

list_len = [len(i) for i in cleaned_tokens_list]
max_len = max(list_len)
print('max_len:', max_len)

X = np.zeros((len(cleaned_tokens_list), max_len))
Y = np.zeros((len(cleaned_tokens_list), ))

for i, tokens in enumerate(cleaned_tokens_list):
    print("i",i)
    print('tokens: ', tokens)
    #tokens, label = tk_lb
    sentence_to_indices(tokens, word_to_index, max_len, i)
    #Y[i] = label
print(X)
print('Data Prepared for model, CPU Time:', time() - start_time)

Removed Noise, CPU Time: 0.000225067138671875
max_len: 14
i 0
tokens:  ['truth', 'tell', "i'd", 'give', '4/5', 'star', 'order', 'recently', 'cheese', 'fry', 'deliver', 'speak', 'anyone', '...']
[[366230. 355345. 372306. 162051.  27552. 341678. 271173. 302802.  97865.
  154596. 120116. 338956.  57187.    868.]]
Data Prepared for model, CPU Time: 0.0019001960754394531


In [ ]:
X[0]

array([366230., 355345., 372306., 162051.,  27552., 341678., 271173.,
       302802.,  97865., 154596., 120116., 338956.,  57187.,    868.])

In [ ]:
api_key='q1HJoEmRMdypie_E6lYwiih0hNfuSatSTQXH6Bh9fnltm30JX4XO1TSCd6NARRVszrs4lMXb7N3i6LWB_UIBEinyG_ah-0expG6GUnzTdUss2gvNQRcJ7fJhK5g2YHYx'
headers = {'Authorization': 'Bearer %s' % api_key}


url = "https://api.yelp.com/v3/businesses/FEVQpbOPOwAPNIgO7D3xxw/reviews"
req = requests.get(url, headers=headers)
print('the status code is {}'.format(req.status_code))


data=json.loads(req.text)
text_to_predict =data["reviews"][0]["text"]
vect = X[0]
from tensorflow import keras
model = keras.models.load_model("/content/gdrive/MyDrive/Etudes/Natural_Language_Processing/SentimentAnalysis/save/")
model.predict(vect)
# I don't understand this output, considering the Y matrix I was supposed to have an array of shape (1,1)  + the final dense layer has a size of 1 ! 

the status code is 200


array([[0.3516037 ],
       [0.37039268],
       [0.4029287 ],
       [0.44840705],
       [0.3950941 ],
       [0.47120404],
       [0.42520157],
       [0.37630567],
       [0.37837526],
       [0.35819   ],
       [0.50418544],
       [0.4415063 ],
       [0.32905757],
       [0.38366202]], dtype=float32)